In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Define Quidditch Teams and their mapped stocks
teams = {
    "Harry Potter": "TSLA",   # Gryffindor 🦁
    "Draco Malfoy": "AAPL",   # Slytherin 🐍
    "Cho Chang": "MSFT",      # Ravenclaw 🦅
    "Cedric Diggory": "AMZN"  # Hufflepuff 🦡
}


In [ ]:
# Download stock data for all teams
data = {}
for team, ticker in teams.items():
    stock_data = yf.download(ticker, start="2023-01-01", end="2025-03-28")
    data[team] = stock_data["Close"]

# Convert data to DataFrame
stock_df = pd.DataFrame(data)
stock_df.to_csv("quidditch_stock_data.csv")
print("Stock data saved.")

In [ ]:
# Data Preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_df)

In [ ]:
# Prepare data for LSTM
seq_length = 10
X, y = [], []
for i in range(len(scaled_data) - seq_length):
    X.append(scaled_data[i:i+seq_length])
    y.append(scaled_data[i+seq_length])
X, y = np.array(X), np.array(y)

In [ ]:
# Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, len(teams))),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(len(teams))
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=20, batch_size=16)

In [ ]:
# Predict Future Trends
future_predictions = model.predict(X[-1].reshape(1, seq_length, len(teams)))
predicted_prices = scaler.inverse_transform(future_predictions)

In [ ]:
# Visualization
plt.figure(figsize=(10, 5))
for idx, team in enumerate(teams.keys()):
    plt.plot(stock_df.index[-30:], stock_df[team][-30:], label=f"{team} Actual", linestyle='dotted')
    plt.scatter(stock_df.index[-1], predicted_prices[0][idx], label=f"{team} Prediction", marker='x')
plt.legend()
plt.title("Seeker Trajectory Prediction (Stock Trends)")
plt.xlabel("Date")
plt.ylabel("Stock Price ($)")
plt.show()
print("LSTM Model trained and predictions visualized.")